# Extração dos dados

## Texto introdutório

O objetivo deste projeto é fazer um modelo preditivo que consiga avaliar quem estará no pódio do prêmio de MVP(Neste projeto considerei os 4 primeiros, mas a NBA considera os 3 primeiros assim como qualquer premiação) e quem será o MVP.

Para isso, irei extrair as estatísticas da temporada regular de todos os jogadores da temporada 1979-80 até a temporada 2020-2021 para fazer o aprendizado do modelo de ML.Neste scraping, extrairei os dados totais,os dados por jogo,os dados por 36 min, os dados por 100 posses ,is dados avançados e os dados do desempenho do time em que o jogador esteve na temporada.Os dados de 2021-2022 serão a entrada para avaliar se o meu modelo consegue acertar os 4 primeiros lugares do prêmio de mvp deste ano.

Além de todas essas colunas que serão geradas na extração,no modelo de dataset proposto por mim, coloco as colunas 'mvp_candidate', 'mvp_podium' e 'mvp_winner'.Estas colunas são valores boleanos para avaliar essas condições.Também coloco a coluna 'mvp_votes_share'(coluna que armazena número de pontos recebidos dividido por número de pontos máximo.Caso o jogador não seja candidato a mvp, esse valor é 0).

As colunas descritas nos parágrafos acima tem como objetivo tentar montar um dataset em que seja possível usar a regressão linear para fazer a predição.Uso como valor de "chave" nomejogador_temporada_time para conseguir diferenciar apropiadamente os jogadores entre si e os valores numéricos são todos fornecidos pelos dados estatísticos.O objetivo é tentar prever o dado numérico de 'mvp_votes_share' de forma que a ordem seja a mesma do prêmio.

Obs:Tenho impressão de que vai haver mvp_votes_share negativos mesmo com uma filtragem, mas espero que ao menos dê para acertar o pódio

Neste notebook extrairei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA.

## Importações e variáveis padrão

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import pandas as pd

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'}

#Necessário para ter as siglas dos times da NBA
times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK',
        'Charlotte Bobcats' : 'CHA',
        'Vancouver Grizzlies' : 'VAN',
        'New Orleans Hornets' : 'NOH',
        'New Orleans/Oklahoma City Hornets' : 'NOK'}

#Intervalos de anos dos dados das temporadas que serão extraidos
#anos = list(range(1980,2021))
anos = [2021]
todos_jogadores = []
todos_mvps = []

## Funções para pegar os dados de mvps e os dados dos times

In [2]:
def pegar_mvps(url, dados_mvps):
    
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    mvps_html = soup.find('tbody').findAll('tr')
    cont_rank = 1
    for mvp_html in mvps_html:

        dados_mvp = {}
        
        dados_mvp['rank'] = cont_rank
        cont_rank += 1
        
        nome_jogador = mvp_html.find('td', {'data-stat':'player'}).find('a').getText()
        dados_mvp['team'] = mvp_html.find('td', {'data-stat':'team_id'}).getText()
        dados_mvp['award_share'] = float(mvp_html.find('td', {'data-stat':'award_share'}).getText())
        
        dados_mvps[nome_jogador] = dados_mvp


def pegar_times_conferencia(soup, dados_times, sufixo):
    times_html = soup.find('table', id = f'confs_standings_{sufixo}')
    
    #Em algumas páginas o id da tag difere, então estou fazendo essa pequena gambiarra para não dar problema,já que
    #só há dois padrões de id para a tag
    if times_html == None:
        times_html = soup.find('table', id = f'divs_standings_{sufixo}').find('tbody').findAll('tr')
    else:
        times_html = soup.find('table', id = f'confs_standings_{sufixo}').find('tbody').findAll('tr')
    
    
    for time_html in times_html:
        if(time_html.get('class')[0] != 'thead'):
            dados_time = {}
            nome_time = times[time_html.find('th').find('a').getText()]

            time_stats_html = time_html.findAll('td')

            for stat in time_stats_html:
                try:
                    dados_time[stat.get('data-stat')] = float(stat.getText())
                except ValueError:
                    dados_time[stat.get('data-stat')] = 0

            dados_times[nome_time] = dados_time

#Função apenas para fazer o request e gerar o objeto BeautifulSoup
def pegar_times(url,dados_times):
    try:
        req = Request(url_dados_time, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    print(ano)
    "Pegando os dados da Conferência Leste"
    pegar_times_conferencia(soup,dados_times, 'E')
    "Pegando os dados da Conferência Oeste"
    pegar_times_conferencia(soup,dados_times, 'W')
    

## Funções para carregar dados dos jogadores

In [3]:
def primeiro_scraping_jogadores(url, dados_jogadores, ano):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
   
    for jogador_html in jogadores_html:
        dados_jogador = {}
        #As linhas da tabela que são da classe thead são meramente quebras entre as linhas,então não as considero
        if jogador_html.get('class')[0] != 'thead':
            
            #Novamente há dois padrões e faço essa pequena gambiarra
            if jogador_html.find('td', {'data-stat':'team_id'}).a == None:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).getText()
            else:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).a.getText()
            
            parametro_player = ''.join(jogador_html.find('td', {'data-stat':'player'}).find('a').getText().split(' '))
            dados_jogador['player_season_team'] = parametro_player +'_' + str(ano) + '_' + sigla_time
            dados_jogador['player'] = jogador_html.find('td', {'data-stat':'player'}).find('a').getText()
            dados_jogador['season'] = ano
            dados_jogador['pos'] = jogador_html.find('td', {'data-stat':'pos'}).getText()
            dados_jogador['age'] = int( jogador_html.find('td', {'data-stat':'age'}).getText() )
            dados_jogador['team'] = sigla_time
            
            for stat in jogador_html.findAll('td')[4:]:
                
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())
                    
                except ValueError:
                    if(stat.getText() != ''):
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    else:
                        dados_jogador[stat.get('data-stat')] = 0.0
            dados_jogador['mp_pergame'] = round(dados_jogador['mp']/dados_jogador['g'], 1)
            dados_jogadores.append(dados_jogador)

        
def intermedirario_scraping_jogadores(url, dados_jogadores):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
    
    i=0
    for jogador_html in jogadores_html:
        dados_jogador = dados_jogadores[i]
       
        if jogador_html.get('class')[0] != 'thead':
            i+=1
            stats_jogador = jogador_html.findAll('td')[8:]
            for stat in stats_jogador:
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())

                except ValueError:
                    if(stat.getText() == ''):
                        dados_jogador[stat.get('data-stat')] = 0.0
                    elif stat.getText()[0] == '-':
                        dados_jogador[stat.get('data-stat')] =(-1)*float('0' + stat.getText()[1:])
                    else:
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    
            

## Bloco principal da extração fazendo um loop por ano

In [4]:
for ano in anos:
    "Nesta seção do código serão carregadas as páginas html"
    
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    url_dados_totais_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_totals.html'
    url_dados_por_jogo_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_game.html'
    url_dados_por_36min_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_minute.html'
    url_dados_por_100pos_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_poss.html'
    url_dados_avancados_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_advanced.html'
    url_votacao_mvp = f'https://www.basketball-reference.com/awards/awards_{ano}.html'
    
    
    "Aqui será carregado os dados dos candidatos a MVP relativos a este ano"
    
    dados_mvps = {}
    pegar_mvps(url_votacao_mvp,dados_mvps)
    
   
    
    "Aqui serão carregados os dados do time"
    
    "Aqui é a variável que armazenará os dados do time relativos àquele ano"
    dados_times = {}
    
    pegar_times(url_dados_time, dados_times)
    
    
    "Finalmente, serão carregados os dados dos jogadores"
    
    dados_jogadores = []
    
    #Pegando os dados de cada página
    
    primeiro_scraping_jogadores(url_dados_totais_jogador, dados_jogadores, int(ano))
    
    intermedirario_scraping_jogadores(url_dados_por_jogo_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_36min_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_100pos_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_avancados_jogador, dados_jogadores)
    
    #As chaves são justamente os nomes dos candidatos a MVP.Uso isso para ficar mais fácil verificar quem é candidato ou não
    nomes_dos_mvps = list(dados_mvps.keys())
    
    #Como o recorte é da década de 80 para cá, e o Lakers não mudou de nome desde esse ano, estou usando essa pequena gambiarra
    #para pegar as colunas dos times
    colunas_time = list(dados_times['LAL'].keys())
    
    for dados_jogador in dados_jogadores:
        sigla_time = dados_jogador['team']
        
        #TOT no basketball reference significa jogadores que jogaram em mais de um time.
        #Há pouquissimos jogadores nessa situação que foram candidatos a MVP e nenhum que ganhou o prêmio.
        if sigla_time != 'TOT': 
            
            if sigla_time == 'CHH':
                sigla_time = 'CHO'
            
            dict_time = dados_times[sigla_time]

            for coluna_time in colunas_time:
                
                dados_jogador['team_' + coluna_time] = dict_time[coluna_time]

            #Verifica se o jogador está entre os candidatos a MVP deste ano
            if dados_jogador['player'] in nomes_dos_mvps:
                dict_mvp = dados_mvps[dados_jogador['player']]
                    
                dados_jogador['mvp_candidate'] = True
                rank_mvp = dict_mvp['rank']


                if(rank_mvp<=4):
                    dados_jogador['mvp_podium'] = True
                    if(rank_mvp==1):
                        dados_jogador['mvp_winner'] = True
                    else:
                        dados_jogador['mvp_winner'] = False
                else:
                    dados_jogador['mvp_podium'] = False
                    dados_jogador['mvp_winner'] = False

                dados_jogador['mvp_votes_share'] = dict_mvp['award_share']

                parametro_player = ''.join(dados_jogador['player'].split(' '))

                parametro_plt = parametro_player + '_' + str(ano) + '_' + dados_jogador['team']
                todos_mvps.append({'player_season_team': parametro_plt,'season':ano ,'rank': rank_mvp,
                                   'mvp_votes_share': dict_mvp['award_share'], 'mvp_candidate':dados_jogador['mvp_candidate'],
                                   'mvp_podium':dados_jogador['mvp_podium'], 'mvp_winner':dados_jogador['mvp_winner']})

            else:
                dados_jogador['mvp_candidate'] = False
                dados_jogador['mvp_podium'] = False
                dados_jogador['mvp_winner'] = False
                dados_jogador['mvp_votes_share'] = 0.0
            

    todos_jogadores += dados_jogadores

2021


In [5]:
dataset1 = pd.DataFrame(todos_jogadores)
dataset1

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,PreciousAchiuwa_2021_MIA,Precious Achiuwa,2021,PF,21,MIA,61,4,737,124,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
1,JaylenAdams_2021_MIL,Jaylen Adams,2021,PG,24,MIL,7,0,18,1,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.0
2,StevenAdams_2021_NOP,Steven Adams,2021,C,27,NOP,58,58,1605,189,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,0.0
3,BamAdebayo_2021_MIA,Bam Adebayo,2021,C,23,MIA,64,64,2143,456,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
4,LaMarcusAldridge_2021_TOT,LaMarcus Aldridge,2021,C,35,TOT,26,23,674,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,DelonWright_2021_SAC,Delon Wright,2021,PG,28,SAC,27,8,696,104,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,0.0
701,ThaddeusYoung_2021_CHI,Thaddeus Young,2021,PF,32,CHI,68,23,1652,370,...,41.0,0.431,18.0,110.7,111.6,-0.94,False,False,False,0.0
702,TraeYoung_2021_ATL,Trae Young,2021,PG,22,ATL,63,63,2125,487,...,31.0,0.569,8.0,113.7,111.4,2.14,False,False,False,0.0
703,CodyZeller_2021_CHO,Cody Zeller,2021,C,28,CHO,48,21,1005,181,...,39.0,0.458,16.0,109.5,111.4,-1.94,False,False,False,0.0


In [6]:
dataset2 = pd.DataFrame(todos_mvps)
dataset2

,player_season_team,season,rank,mvp_votes_share,mvp_candidate,mvp_podium,mvp_winner
0,GiannisAntetokounmpo_2021_MIL,2021,4,0.345,True,True,False
1,StephenCurry_2021_GSW,2021,3,0.449,True,True,False
2,LukaDončić_2021_DAL,2021,6,0.042,True,False,False
3,JoelEmbiid_2021_PHI,2021,2,0.580,True,True,False
4,RudyGobert_2021_UTA,2021,10,0.008,True,False,False
5,JamesHarden_2021_HOU,2021,13,0.001,True,False,False
6,JamesHarden_2021_BRK,2021,13,0.001,True,False,False
7,LeBronJames_2021_LAL,2021,14,0.001,True,False,False
8,NikolaJokić_2021_DEN,2021,1,0.961,True,True,True
9,KawhiLeonard_2021_LAC,2021,15,0.001,True,False,False


In [7]:
dataset1.to_csv('./data/dataset_jogadores_2021.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset2.to_csv('./data/dataset_mvps_2021.csv', sep=';', index = False, encoding = 'utf-8-sig')

## Fazendo um tratamento básico

In [8]:
import pandas as pd

dataset_jogadores = pd.read_csv('./data/dataset_jogadores_2021.csv', sep=';')

dataset_jogadores.head()

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,PreciousAchiuwa_2021_MIA,Precious Achiuwa,2021,PF,21,MIA,61,4,737,124,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
1,JaylenAdams_2021_MIL,Jaylen Adams,2021,PG,24,MIL,7,0,18,1,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.0
2,StevenAdams_2021_NOP,Steven Adams,2021,C,27,NOP,58,58,1605,189,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,0.0
3,BamAdebayo_2021_MIA,Bam Adebayo,2021,C,23,MIA,64,64,2143,456,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
4,LaMarcusAldridge_2021_TOT,LaMarcus Aldridge,2021,C,35,TOT,26,23,674,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
for coluna in dataset_jogadores.columns:
    print(coluna)

player_season_team
player
season
pos
age
team
g
gs
mp
fg
fga
fg_pct
fg3
fg3a
fg3_pct
fg2
fg2a
fg2_pct
efg_pct
ft
fta
ft_pct
orb
drb
trb
ast
stl
blk
tov
pf
pts
mp_pergame
fga_per_g
fg3_per_g
fg3a_per_g
fg2_per_g
fg2a_per_g
ft_per_g
fta_per_g
orb_per_g
drb_per_g
trb_per_g
ast_per_g
stl_per_g
blk_per_g
tov_per_g
pf_per_g
pts_per_g
fga_per_mp
fg3_per_mp
fg3a_per_mp
fg2_per_mp
fg2a_per_mp
ft_per_mp
fta_per_mp
orb_per_mp
drb_per_mp
trb_per_mp
ast_per_mp
stl_per_mp
blk_per_mp
tov_per_mp
pf_per_mp
pts_per_mp
fga_per_poss
fg3_per_poss
fg3a_per_poss
fg2_per_poss
fg2a_per_poss
ft_per_poss
fta_per_poss
orb_per_poss
drb_per_poss
trb_per_poss
ast_per_poss
stl_per_poss
blk_per_poss
tov_per_poss
pf_per_poss
pts_per_poss
Unnamed: 80
off_rtg
def_rtg
fg3a_per_fga_pct
fta_per_fga_pct
orb_pct
drb_pct
trb_pct
ast_pct
stl_pct
blk_pct
tov_pct
usg_pct
DUMMY
ows
dws
ws
ws_per_48
obpm
dbpm
bpm
vorp
team_wins
team_losses
team_win_loss_pct
team_gb
team_pts_per_g
team_opp_pts_per_g
team_srs
mvp_candidate
mvp_podium

### As duas colunas abaixo  são de colunas vazias do basketball reference, então elas devem ser apagadas

In [10]:
dataset_jogadores['Unnamed: 80']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
700    0.0
701    0.0
702    0.0
703    0.0
704    0.0
Name: Unnamed: 80, Length: 705, dtype: float64

In [11]:
dataset_jogadores['DUMMY']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
700    0.0
701    0.0
702    0.0
703    0.0
704    0.0
Name: DUMMY, Length: 705, dtype: float64

In [12]:
dataset_jogadores.drop(columns = ['Unnamed: 80', 'DUMMY'], inplace = True)
dataset_jogadores.head()

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,PreciousAchiuwa_2021_MIA,Precious Achiuwa,2021,PF,21,MIA,61,4,737,124,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
1,JaylenAdams_2021_MIL,Jaylen Adams,2021,PG,24,MIL,7,0,18,1,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.0
2,StevenAdams_2021_NOP,Steven Adams,2021,C,27,NOP,58,58,1605,189,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,0.0
3,BamAdebayo_2021_MIA,Bam Adebayo,2021,C,23,MIA,64,64,2143,456,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.0
4,LaMarcusAldridge_2021_TOT,LaMarcus Aldridge,2021,C,35,TOT,26,23,674,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Salvando após as duas remoções

In [13]:
dataset_jogadores.to_csv('./data/dataset_jogadores_2021.csv', sep=';', index = False, encoding = 'utf-8-sig')

## Cenas para os próximos capitulos

In [14]:
#Seleção de jogadores que jogaram menos de 20 min por média e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['mp_pergame'] < 20)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share


In [15]:
#Seleção de jogadores que jogaram menos de 40% das partidas da temporada e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['g']/(dataset_jogadores['team_wins'] + dataset_jogadores['team_losses']) < 0.4)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
259,JamesHarden_2021_HOU,James Harden,2021,SG,31,HOU,8,8,290,60,...,55.0,0.236,35.0,108.8,116.7,-7.50,True,False,False,0.001
571,DerrickRose_2021_DET,Derrick Rose,2021,PG,32,DET,15,0,342,79,...,52.0,0.278,29.0,106.6,111.1,-4.38,True,False,False,0.010


In [16]:
#Seleção de jogadores que jogaram em times com %wins_losses menor que 40% na temporada e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['team_win_loss_pct'] < 0.39)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
259,JamesHarden_2021_HOU,James Harden,2021,SG,31,HOU,8,8,290,60,...,55.0,0.236,35.0,108.8,116.7,-7.50,True,False,False,0.001
571,DerrickRose_2021_DET,Derrick Rose,2021,PG,32,DET,15,0,342,79,...,52.0,0.278,29.0,106.6,111.1,-4.38,True,False,False,0.010
